In [2]:
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

import backtrader as bt


src_path = '/Users/yifeining/Documents/GitHub/triplec-quant-trading/src'
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    
from models import *
from strategy import *

In [28]:

cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(Strategy)

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
dir_path = '/Users/yifeining/Documents/GitHub/triplec-quant-trading'
# datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')
datapath = os.path.join(dir_path, 'data', 'survivorship-free', 'AAPL.csv')

# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
    dataname=datapath,
    # Do not pass values before this date
    fromdate=datetime.datetime(2013, 1, 1),
    # Do not pass values before this date
    todate=datetime.datetime(2015, 12, 31),
    # Do not pass values after this date
    reverse=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(10000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.001)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00
